In [1]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment, indent

import numpy as np
import os

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, encoding="ISO-8859-1")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [53]:
## Parameters
# data folder
source_folder = r'E:\Data\20231006-KPtest0615_MF2'
# target saving folder on the storage server
target_drive = r'\\wi-bigdata\weissman_imaging\puzheng\4T1_tumor'
# HAL imaging protocol
imaging_protocol = r'Conv_zscan_748_637_477_s13_n1000_5Hz'
# HAL imaging protocol in the reference round, usually Hyb 0
ref_hyb = 0
ref_imaging_protocol = r'Conv_zscan_748_637_477_405_s13_n1000_5Hz'

In [54]:
## Paramerters 2: change according to experiment design:
valve_start = 0

hyb_start = 0

reg_start = 1

num_rounds = 14 + 1 + 1

#data_type = 'R'
data_type = 'M'

valve_ids = np.arange(valve_start, valve_start + num_rounds)
hyb_ids = np.arange(hyb_start, hyb_start + num_rounds)
folder_names = [f'H{_h}{data_type}{_r}' for _h, _r in zip(np.arange(hyb_start, hyb_start + num_rounds), 
                                                     np.arange(reg_start, reg_start + num_rounds))]
print(folder_names)

['H0M1', 'H1M2', 'H2M3', 'H3M4', 'H4M5', 'H5M6', 'H6M7', 'H7M8', 'H8M9', 'H9M10', 'H10M11', 'H11M12', 'H12M13', 'H13M14', 'H14M15', 'H15M16']


In [55]:
recipe = Element('recipe')
cmd_seq = SubElement(recipe, 'command_sequence')

copy_data = False

for _i, (_vid, _hid, _folder) in enumerate(zip(valve_ids, hyb_ids, folder_names)):

    if _hid == ref_hyb:
        # determine ref_imaging protocol
        _im_p = ref_imaging_protocol
        # add hyb 0
        # comments
        comment = Comment(f"Ref Hyb {_hid} in folder: {_folder}")
        cmd_seq.append(comment)

    else:
        _im_p = imaging_protocol
        # Add TCEP
        if _i > 0:
            # TCEP
            tcep = SubElement(cmd_seq, 'valve_protocol')
            tcep.text = "Flow TCEP"
            # wash tcep
            tcep_wash = SubElement(cmd_seq, 'valve_protocol')
            tcep_wash.text = "Flow Wash"
            indent(tcep_wash)
        # comments
        comment = Comment(f"Hyb {_hid} with tube:{_vid} in folder: {_folder}")
        cmd_seq.append(comment)
        # Adaptor
        adt = SubElement(cmd_seq, 'valve_protocol')
        adt.text = f"Flow Hyb {_vid}"
        # delay time
        adt_incubation = SubElement(cmd_seq, 'valve_protocol')
        adt_incubation.text = f"Incubate 15min"
        # wash adaptors
        adt_wash = SubElement(cmd_seq, 'valve_protocol')
        adt_wash.text = "Flow Wash"
        
        # Readouts
        readouts = SubElement(cmd_seq, 'valve_protocol')
        readouts.text = "Flow Common Readouts"
        # incubate readouts
        readout_incubation = SubElement(cmd_seq, 'valve_protocol')
        readout_incubation.text = f"Incubate 15min"
        # wash readouts
        readout_wash = SubElement(cmd_seq, 'valve_protocol')
        readout_wash.text = f"Flow Wash"
        
    ## For all rounds, imaging buffer and imaging loop and data copy is required:
    # flow imaging buffer
    imaging = SubElement(cmd_seq, 'valve_protocol')
    imaging.text = f"Flow Imaging"
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, _folder)
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"
    # imaging loop    
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=_im_p)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "2000"
    if copy_data:
    # copy folder
        copy_dir = SubElement(cmd_seq, 'copy_directory')
        source_dir = SubElement(copy_dir, 'source_path')
        source_dir.text = change_dir.text 
        target_dir = SubElement(copy_dir, 'target_path')
        target_dir.text = os.path.join(target_drive, 
                                       os.path.basename(os.path.dirname(source_dir.text)), 
                                       os.path.basename(source_dir.text))
        del_source = SubElement(copy_dir, 'delete_source')
        del_source.text = "True"

ssc = SubElement(cmd_seq, 'valve_protocol')
ssc.text = f"Flow 2xSSC"

#final_str = prettify(cmd_seq)    
#print( final_str )

In [56]:
# add supplementary scripts
import re
protocol = imaging_protocol
channels = [_s for _s in protocol.split('_') if re.match('^[0-9]+$', _s, )]
steps = int(re.search(r'_s([0-9]+)_', protocol).groups()[0])

hal_item = SubElement(recipe, 'item', name=f"{protocol}")
movie = SubElement(hal_item, 'movie')
name = SubElement(movie, 'name', increment="Yes")
name.text = f"Conv_zscan"
overwrite = SubElement(movie, 'overwrite')
overwrite.text = "True"
length = SubElement(movie, 'length')
length.text = f"{steps * len(channels)}"
parameters = SubElement(movie, 'parameters')
parameters.text = f"{protocol}"
delay = SubElement(movie, 'delay')
delay.text = f"{500}"
check_focus = SubElement(movie, 'check_focus')
num_focus_checks = SubElement(check_focus, 'num_focus_checks')
num_focus_checks.text = f"{100}"
focus_scan = SubElement(check_focus, 'focus_scan')
variable_entry = SubElement(movie, 'variable_entry', name= "Position Loop Zscan")
variable_entry.text = f"{protocol}"

In [57]:
# add supplementary scripts
import re
protocol = ref_imaging_protocol
channels = [_s for _s in protocol.split('_') if re.match('^[0-9]+$', _s, )]
steps = int(re.search(r'_s([0-9]+)_', protocol).groups()[0])

hal_item = SubElement(recipe, 'item', name=f"{protocol}")
movie = SubElement(hal_item, 'movie')
name = SubElement(movie, 'name', increment="Yes")
name.text = f"Conv_zscan"
overwrite = SubElement(movie, 'overwrite')
overwrite.text = "True"
length = SubElement(movie, 'length')
length.text = f"{steps * len(channels)}"
parameters = SubElement(movie, 'parameters')
parameters.text = f"{protocol}"
delay = SubElement(movie, 'delay')
delay.text = f"{500}"
check_focus = SubElement(movie, 'check_focus')
num_focus_checks = SubElement(check_focus, 'num_focus_checks')
num_focus_checks.text = f"{100}"
focus_scan = SubElement(check_focus, 'focus_scan')
variable_entry = SubElement(movie, 'variable_entry', name= "Position Loop Zscan")
variable_entry.text = f"{protocol}"

In [58]:
# add loop
loop_variable = SubElement(recipe, 'loop_variable', name=f"Position Loop Zscan")
file_path = SubElement(loop_variable, 'file_path')
file_path.text = "positions_all.txt"

In [59]:
final_str = prettify(recipe)

In [60]:
print(final_str)

<?xml version="1.0" ?>
<recipe>
	<command_sequence>
		<!--Ref Hyb 0 in folder: H0M1-->
		<valve_protocol>Flow Imaging</valve_protocol>
		<change_directory>E:\Data\20231006-KPtest0615_MF2\H0M1</change_directory>
		<wakeup>5000</wakeup>
		<loop name="Position Loop Zscan" increment="name">
			<item name="Conv_zscan_748_637_477_405_s13_n1000_5Hz"> </item>
		</loop>
		<delay>2000</delay>
		<valve_protocol>Flow TCEP</valve_protocol>
		<valve_protocol>Flow Wash</valve_protocol>
		<!--Hyb 1 with tube:1 in folder: H1M2-->
		<valve_protocol>Flow Hyb 1</valve_protocol>
		<valve_protocol>Incubate 15min</valve_protocol>
		<valve_protocol>Flow Wash</valve_protocol>
		<valve_protocol>Flow Common Readouts</valve_protocol>
		<valve_protocol>Incubate 15min</valve_protocol>
		<valve_protocol>Flow Wash</valve_protocol>
		<valve_protocol>Flow Imaging</valve_protocol>
		<change_directory>E:\Data\20231006-KPtest0615_MF2\H1M2</change_directory>
		<wakeup>5000</wakeup>
		<loop name="Position Loop Zscan" increm

In [61]:
if not os.path.exists(source_folder):
    print(f"Create folder: {source_folder}")
    os.makedirs(source_folder)
    
save_filename = os.path.join(source_folder, f"generated_dave_H{hyb_start}-{hyb_start+num_rounds-1}.xml")
with open(save_filename, 'w') as _output_handle:
    print(f"Dave protocol saved to file: {save_filename}")
    _output_handle.write(final_str)

Dave protocol saved to file: E:\Data\20231006-KPtest0615_MF2\generated_dave_H0-15.xml
